In [ ]:
import numpy as np
import astropy.io.fits as fits
import pyklip.parallelized
import pyklip.instruments.Instrument as Instrument
import pyklip.fakes

import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
with fits.open("Project_Materials_2009/center_im.fits") as hdulist:
    img_cube = hdulist[0].data # Series of images of the star system taken in time 
    exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']
    
with fits.open("Project_Materials_2009/median_unsat.fits") as hdulist:
    calib_frame = hdulist[0].data # 1024 x 1024 image of the unsaturated star for photometric calibration
    calib_exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']
    
with fits.open("Project_Materials_2009/rotnorth.fits") as hdulist:
    rot_angles = hdulist[0].data # rotation angle of the night sky for each science frame
    
plt.figure()
plt.imshow(img_cube[0], cmap="inferno")
plt.xlim([512-150, 512+150])
plt.ylim([512-150, 512+150])
plt.title("Science Frame")

plt.figure()
plt.imshow(calib_frame, cmap="inferno")
plt.xlim([512-150, 512+150])
plt.ylim([512-150, 512+150])
plt.title("Calibration Frame")

In [ ]:
centers = np.array([[(img_cube.shape[2]-1)/2, (img_cube.shape[1]-1)/2] for _ in range(img_cube.shape[0])])

dataset = Instrument.GenericData(img_cube, centers, parangs=-rot_angles)
dataset.OWA = 250

In [ ]:
pyklip.parallelized.klip_dataset(dataset, outputdir="./", fileprefix="epoch1", annuli=11, 
                                 subsections=1, numbasis=[20], maxnumbasis=100, mode="ADI",
                                 movement=1)

In [ ]:
output_img = np.nanmean(dataset.output[0,:,0], axis=0)

plt.figure()
plt.imshow(output_img, cmap="inferno", vmin=np.nanpercentile(output_img, 1), vmax=np.nanpercentile(output_img, 99.7))
plt.xlim([512-200, 512+200])
plt.ylim([512-200, 512+200])
plt.title("Stellar PSF Subtracted Image")

In [ ]:
guess_x, guess_y = 575, 583
guess_flux = 800

peakflux, fwhm, planet_x, planet_y = pyklip.fakes.gaussfit2d(output_img, guess_x, guess_y, guesspeak=guess_flux)
print(peakflux, planet_x, planet_y)

star_y, star_x = dataset.output_centers[0]

planet_sep_pixels = np.sqrt((planet_x - star_x)**2 + (planet_y - star_y)**2)
planet_PA = np.degrees(np.arctan2(-(planet_x - star_x), planet_y - star_y)) % 360

platescale = 9.952 # mas/pixel
planet_sep_mas = planet_sep_pixels * platescale

print(planet_sep_mas, planet_PA)

In [ ]:
peakflux_star, _, _, _ = pyklip.fakes.gaussfit2d(calib_frame, 511.5, 511.5, guesspeak=10000)

flux_ratio = (peakflux/exptime)/(peakflux_star/calib_exptime)
print(flux_ratio)